In [32]:
from torch import nn
import torch
import numpy as np
from Unet import UNet

In [7]:
model = UNet(in_channels=3, out_channels=3)

In [124]:
class Diffusion(nn.Module):
    def __init__(self,config,
                score_model,):
        super().__init__()
        
        self.config = config
        self.score_model = model
        self.T = config['Time']
        self.M = 10
        self.alpha =np.sqrt(1 - 0.02 * np.arange(1, self.T + 1) / self.T)
        self.beta = np.sqrt(1 - self.alpha**2)
        self.bar_alpha = np.cumprod(self.alpha)
        self.bar_beta = np.sqrt(1 - self.bar_alpha**2)
        self.sigma = self.beta.copy()
    def forward(self,x_t,t):
        
        return self.score_model(x_t,t)
    
    def forward_train(self,x_0,t):
        output = self.margin_pt(x_0,t)
        shape = x_0.shape
        z= self.target_distribut(*shape)
        label = self.creat_label(z)
        loss =self.model.loss(output,label)
        return loss
    def margin_pt(self,x_0,t):

        shape = x_0.shape
        z = torch.rand(*shape)
        x_t  = np.sqrt(self.bar_beta[t])*x_0 + z*np.sqrt(1-self.bar_beta[t])  
        return x_t 
    
    def noise(self,x_0):
        shape = x_0.shape
        pre_t= 0
        for t in range(1,self.T+1):
            theta = torch.rand(*shape)
            x_0 +=self.f_x_t(x_0,t)*(t-pre_t) +np.sqrt(self.g_t(t))*noise
            pre_t = t
        return x_0
    
    
    def denoise(self,x_T):
        shape = x_T.shape
        pre_t= self.T
        for t in reversed(range(self.T)):
            z = torch.rand(*shape)
            
            gtdw = np.sqrt(self.g_t(t))*z
            x_T +=(self.f_x_t(x_T,t)-self.g_t(t)*self.g_t(t)*self.score_model(x_T,t))*(pre_t-t) +gtdw
            pre_t = t
            
            score = self.score_model(x_T,t)
            for j in range(1,self.M):
                z = torch.rand(*shape)
                x_T +=self.sigma[t]*score +torch.sqrt(2*self.sigma[i]* z)
            
        return x_T
    
    def denoise_ode(self,x_T):
        shape = x_T.shape
        pre_t= self.T
        for t in reversed(range(self.T)):
            theta = torch.rand(*shape)
            x_T +=(self.f_x_t(x,t)-self.g_t(t)*self.g_t(t)*self.score_model(x_T,t))*(pre_t-t) 
            pre_t = t
            for j in range(1,self.M):
                theta = torch.rand(*shape)
        return x_T
    def f_x_t(self,x,t):
        return 0
    def g_t(self,t):
        return 0
    def creat_label(self,noise):
        return noise
    def target_distribut(self,*shape):
        return torch.rand(shape)

In [125]:
#模型自选
#DDPM
def ddpm_f_x_t(self,x,t):
    return -0.5*x* self.beta[t]
def ddpm_g_t(self,t):
    return np.sqrt(self.beta[t])
Diffusion.f_x_t=ddpm_f_x_t
Diffusion.g_t= ddpm_g_t
ddpm =Diffusion({'Time':2},model)


In [132]:
#SMDL
def smdl_f_x_t(self,x,t):
    return 0
def smdl_g_t(self,t):
    ratio = self.config['theta_max']/self.config['theta_min']
    weight=self.config['theta_min']*(ratio)**(t/self.config['Time'])
    part = np.sqrt(2*np.log(ratio))
    return weight*part
Diffusion.f_x_t=smdl_f_x_t
Diffusion.g_t= smdl_g_t
diff_model =Diffusion({'Time':2,'theta_max':0.1,'theta_min':0.001},model)

In [133]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=0.99)
learn_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=10)

#计算能力决定数据集的设置
pipe = Compose([transforms.Resize([224,224]),
               transforms.ToTensor(),
               ])
dataset = ImageFolder(path,
                      transform=pipe,)
dataloader = DataLoader(dataset,
                        batch_size=4,
                        shuffle=True,
                        pin_memory=True)

0.03034854258770293

In [ ]:
epoches = 20

for i in range(epoches):
    
    for data in dataloader:
        image,_= data
        bs=image.shape[0]
        optimizer.zero_grad()
        t = torch.randint(diff_model.config['Time'],(bs,1,1,1))
        noise = torch.rand(*image.shape)
        
        output = diff_model(noise,t)
        label = diff_model.creat_label(noise)
        loss = diff_model.score_model.loss(output,label)
        
        loss.backward()
        optimizer.step()
        learn_scheduler.step()
        print('finish one iter')# 具体需要可视化讯息可以设置
    